In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [90]:
def load_data(file):
    with open(file,'r', encoding='utf-8') as f:
        data = pd.read_csv(f)
    return (data)
        

In [91]:
path='/kaggle/input/nlp-getting-started/test.csv'

In [92]:
data = load_data(path)

In [93]:
data.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [94]:


df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df.head(1)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1


In [95]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [96]:
df.shape

(7613, 5)

In [97]:
df = df[['text','target']]

In [98]:
df.iloc[0]["text"]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [99]:
df.head(5)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [100]:
nlp = spacy.load('en_core_web_sm')
#from spacy.lang.en import English
#nlp = English()
#nlp = spacy.blank('en')
#nlp.pipe_names

In [101]:
# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.add_pipe('textcat',last=True)

In [102]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'textcat']

In [103]:
# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.create_pipe("textcat")

In [104]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'textcat']

In [105]:
# Adding the labels to textcat
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

1

In [106]:
textcat.labels

('POSITIVE', 'NEGATIVE')

In [107]:
# Converting review text to tuple 
df['tuples'] = df.apply(lambda row: (row['text'],row['target']), axis=1)

In [108]:
df.head(5)

,text,target,tuples
0,Our Deeds are the Reason of this #earthquake M...,1,(Our Deeds are the Reason of this #earthquake ...
1,Forest fire near La Ronge Sask. Canada,1,"(Forest fire near La Ronge Sask. Canada, 1)"
2,All residents asked to 'shelter in place' are ...,1,(All residents asked to 'shelter in place' are...
3,"13,000 people receive #wildfires evacuation or...",1,"(13,000 people receive #wildfires evacuation o..."
4,Just got sent this photo from Ruby #Alaska as ...,1,(Just got sent this photo from Ruby #Alaska as...


In [109]:
# This is how data looks like 
df["tuples"][0]

('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', 1)

In [110]:
# Converting tuple to List 
train = df['tuples'].tolist()

In [111]:
print(train[4])
#print(len(train))

('Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ', 1)


In [112]:
texts, labels = zip(*train)

In [113]:
texts[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [114]:
labels[0]

1

In [115]:
#what we did is bascially created Flags if its Positive aka True and negative we gave False
cats = []
for y in labels:
    if(bool(y)):
        cats.append({"POSITIVE": True, "NEGATIVE":False})
    else:
        cats.append({"POSITIVE": False, "NEGATIVE":True})

In [116]:
TrainX = texts
TrainY = cats

In [117]:
len(TrainX)

7613

In [118]:
len(TrainY)

7613

In [119]:
#training data has to be in this format
train_data = list(zip(TrainX,[{'cats': cats} for cats in TrainY]))
train_data[0]

('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 {'cats': {'POSITIVE': True, 'NEGATIVE': False}})

In [120]:
len(train_data)

7613

Model

In [121]:
n_iter=20

In [122]:
from thinc.api import Adam

optimizer = Adam(
    learn_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    eps=1e-08,
    L2=1e-6,
    grad_clip=1.0,
    use_averages=True,
    L2_is_weight_decay=True
)

In [123]:
from spacy.util import minibatch, compounding
from spacy.training.example import Example
# Construction from subclass
from spacy.lang.en import English
nlp = English()

# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.add_pipe('textcat',last=True)

# Adding the labels to textcat
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

# Converting review text to tuple 
df['tuples'] = df.apply(lambda row: (row['text'],row['target']), axis=1)

# Converting tuple to List 
train = df['tuples'].tolist()
texts, labels = zip(*train)

#what we did is bascially created Flags if its Positive aka True and negative we gave False
cats = []
for y in labels:
    if(bool(y)):
        cats.append({"POSITIVE": True, "NEGATIVE":False})
    else:
        cats.append({"POSITIVE": False, "NEGATIVE":True})
        
TrainX = texts
TrainY = cats

#training data has to be in this format
train_data = list(zip(TrainX,[{'cats': cats} for cats in TrainY]))

# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    #optimizer = optimizer
    print("Training the model...")
    
    # Performing training
    for i in range(n_iter):
        print("Epoch : {} ".format(i))
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:

            texts, annotations = zip(*batch)
                
            example = []
                # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
                
                # Update the model
            nlp.update(example, drop=0.5, losses=losses)

[2023-06-07 23:28:30,919] [INFO] Created vocabulary
[2023-06-07 23:28:30,928] [INFO] Finished initializing nlp object


Training the model...
Epoch : 0 
Epoch : 1 
Epoch : 2 
Epoch : 3 
Epoch : 4 
Epoch : 5 
Epoch : 6 
Epoch : 7 
Epoch : 8 
Epoch : 9 
Epoch : 10 
Epoch : 11 
Epoch : 12 
Epoch : 13 
Epoch : 14 
Epoch : 15 
Epoch : 16 
Epoch : 17 
Epoch : 18 
Epoch : 19 


In [124]:
nlp.to_disk("sentiment")

In [125]:
nlp = spacy.load("sentiment")

In [126]:
test_data = load_data(path)
# Converting test data to List 
test = test_data['text'].tolist()

In [127]:
len(test)

3263

In [128]:
test[0]

'Just happened a terrible car crash'

In [129]:
# Testing the model
doc=nlp(test[0])
doc.cats 

{'POSITIVE': 0.9999884366989136, 'NEGATIVE': 1.158378927357262e-05}

In [130]:
test[3]

'Apocalypse lighting. #Spokane #wildfires'

In [131]:
# Testing the model
doc=nlp(test[3])
doc.cats 

{'POSITIVE': 0.9999996423721313, 'NEGATIVE': 3.581536418550968e-07}

In [132]:
test[4]

'Typhoon Soudelor kills 28 in China and Taiwan'

In [133]:
# Testing the model
#test[4]
doc=nlp(test[4])
doc.cats 

{'POSITIVE': 0.9999996423721313, 'NEGATIVE': 4.161551601100655e-07}